In [ ]:
from owslib.wfs import WebFeatureService

In [ ]:
wfs11 = WebFeatureService(url='https://entw-imis.lab.bfs.de/ogc/opendata/wfs', version='1.1.0')
wfs11.identification.title

In [ ]:
[operation.name for operation in wfs11.operations]

In [ ]:
list(wfs11.contents)

In [3]:
#opendata:public_precipitation_15min
#opendata:public_odl_brutto_1h
typeName='opendata:public_odl_brutto_1h'

In [ ]:
wfs11.get_schema(typeName)

In [ ]:
import xml.dom.minidom # WFS returns GML - this library is used to parse it

response = wfs11.getfeature(typename=typeName, bbox=(7.800293,47.709762,13.579102,54.033586), srsname='EPSG:4326',maxfeatures=2)
xml = xml.dom.minidom.parse(response) # or xml.dom.minidom.parseString(xml_string)
print(xml.toprettyxml(indent="    "))

In [ ]:
from owslib.fes import *
from owslib.etree import etree
import xml.dom.minidom # WFS returns GML - this library is used to parse it

# apply filter by property name and value
filter = property
filter=PropertyIsGreaterThanOrEqualTo(propertyname='opendata:end_measure', literal='2021-07-29T00:00:000Z')
filterxml = etree.tostring(filter.toXML()).decode("utf-8")

response = wfs11.getfeature(typename=typeName,filter = filterxml, maxfeatures=5)

xml = xml.dom.minidom.parse(response) # or xml.dom.minidom.parseString(xml_string)
print(xml.toprettyxml(indent="    "))

In [ ]:
import json

#7.089845,45.253298,12.166505,55.075436
#7.800293,47.709762,13.579102,54.033586
response1 = wfs11.getfeature(typename='opendata:nuklide_fisch',bbox=(7.800293,47.709762,13.579102,54.033586), srsname='EPSG:4326', maxfeatures=2, outputFormat='application%2Fjson')
# convert IO-byte to bytes
bytesD=bytes(response1.read())
# convert to json
data = json.loads(bytesD)
print(data)

In [ ]:
from owslib.fes import *
from owslib.etree import etree
import json
import pandas as pd

filter= PropertyIsLike(propertyname='id', literal='DEZ2408',wildCard='*')
#filter2 = PropertyIsGreaterThanOrEqualTo(propertyname='end_measure', literal='2022-11-17')
#filters=[filter1,filter2]


filterxml = etree.tostring(filter.toXML()).decode("utf-8")
#filterxml = etree.tostring(And(operations=filters).toXML()).decode("utf-8")

response = wfs11.getfeature(typename='opendata:odlinfo_timeseries_precipitation_15min',filter=filterxml,bbox=(7.800293,47.709762,13.579102,54.033586), maxfeatures=10, srsname='EPSG:4326',outputFormat='application/json')
bytesD=bytes(response.read())
# convert to json
data = json.loads(bytesD)
print(data)


In [ ]:
from owslib.fes import *
from owslib.etree import etree
import json

# apply filter by property name and value

#filterxml = "<ogc:Filter><ogc:PropertyIsGreaterThanOrEqualTo><ogc:PropertyName>end_measure</ogc:PropertyName><ogc:Literal>2022-08-09</ogc:Literal></ogc:PropertyIsGreaterThanOrEqualTo><ogc:PropertyIsEqualTo><ogc:PropertyName>source</ogc:PropertyName><ogc:Literal>BFS</ogc:Literal></ogc:PropertyIsEqualTo></ogc:Filter>"   # create the filter for the specie
filter1 = PropertyIsGreaterThanOrEqualTo(propertyname='end_measure', literal='2022-01-01')
#filter1= PropertyIsNull(propertyname='value')
filter2= PropertyIsLike(propertyname='network', literal='BfS',wildCard='*')
filter3= PropertyIsLike(propertyname='id', literal='DEZ2634',wildCard='*')
#filter3=PropertyIsGreaterThanOrEqualTo(propertyname='height_above_sea', literal=100)
#filter2 = PropertyIsEqualTo(propertyname='network_id', literal='Z')
filters=[filter1,filter2,filter3]




#filterxml = '<ogc:Filter><ogc:And><ogc:PropertyIsGreaterThanOrEqualTo xmlns:ogc="http://www.opengis.net/ogc"><ogc:PropertyName>sample_end</ogc:PropertyName><ogc:Literal>2022-07-12</ogc:Literal></ogc:PropertyIsGreaterThanOrEqualTo><ogc:PropertyIsLike xmlns:ogc="http://www.opengis.net/ogc" wildCard="*" singleChar="_" escapeChar="\\"><ogc:PropertyName>local_authority</ogc:PropertyName><ogc:Literal>Marktbreit</ogc:Literal></ogc:PropertyIsLike></ogc:And></ogc:Filter>'
filterxml = etree.tostring(And(operations=filters).toXML()).decode("utf-8")
#filterxml = etree.tostring(filter1.toXML()).decode("utf-8")

response = wfs11.getfeature(typename=typeName,filter = filterxml,outputFormat='application%2Fjson')
# convert IO-byte to bytes
bytesD=bytes(response.read())
# convert to json
data = json.loads(bytesD)

In [4]:
from owslib.fes import *
from owslib.etree import etree
import json

# apply filter by property name and value

#filterxml = "<ogc:Filter><ogc:PropertyIsGreaterThanOrEqualTo><ogc:PropertyName>end_measure</ogc:PropertyName><ogc:Literal>2022-08-09</ogc:Literal></ogc:PropertyIsGreaterThanOrEqualTo><ogc:PropertyIsEqualTo><ogc:PropertyName>source</ogc:PropertyName><ogc:Literal>BFS</ogc:Literal></ogc:PropertyIsEqualTo></ogc:Filter>"   # create the filter for the specie
#filter1 = PropertyIsGreaterThanOrEqualTo(propertyname='end_measure', literal='2021-01-01')
filter1 = PropertyIsBetween(propertyname='start', lower='2022-01-01' ,upper='2022.12.23 06:00:00')
#filter1= PropertyIsNull(propertyname='value')
#filter2= PropertyIsLike(propertyname='source', literal='BfS',wildCard='*')
filter2= PropertyIsLike(propertyname='id', literal='DEZ0177',wildCard='*')
#filter3=PropertyIsGreaterThanOrEqualTo(propertyname='height_above_sea', literal=100)
#filter2 = PropertyIsEqualTo(propertyname='network_id', literal='Z')
filters=[filter1,filter2]




#filterxml = '<ogc:Filter><ogc:And><ogc:PropertyIsGreaterThanOrEqualTo xmlns:ogc="http://www.opengis.net/ogc"><ogc:PropertyName>sample_end</ogc:PropertyName><ogc:Literal>2022-07-12</ogc:Literal></ogc:PropertyIsGreaterThanOrEqualTo><ogc:PropertyIsLike xmlns:ogc="http://www.opengis.net/ogc" wildCard="*" singleChar="_" escapeChar="\\"><ogc:PropertyName>local_authority</ogc:PropertyName><ogc:Literal>Marktbreit</ogc:Literal></ogc:PropertyIsLike></ogc:And></ogc:Filter>'
filterxml = etree.tostring(And(operations=filters).toXML()).decode("utf-8")
#filterxml = etree.tostring(filter1.toXML()).decode("utf-8")

response = wfs11.getfeature(typename=typeName,filter = filterxml,outputFormat='application%2Fjson')
# convert IO-byte to bytes
bytesD=bytes(response.read())
# convert to json
data = json.loads(bytesD)

In [5]:
#save json to show in qgis
import json
with open(typeName + '.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
#save geojson and add layer in qgis
import os # This is is needed in the pyqgis console also
from qgis.core import (
    QgsVectorLayer
)
from qgis.core import QgsProject

# get the path to the shapefile e.g. /home/project/data/ports.shp
path_to_layer = "geotest.geojson"

# The format is:
# vlayer = QgsVectorLayer(data_source, layer_name, provider_name)

vlayer = QgsVectorLayer(path_to_layer, "bfsgeo", "ogr")
if not vlayer.isValid():
    print("Layer failed to load!")
else:
    QgsProject.instance().addMapLayer(vlayer)

In [ ]:

import pandas as pd
from matplotlib import pyplot as plt
import requests
import json



#df = pd.DataFrame(data=data)
#print(df)

# using pandas for serializing
df= pd.json_normalize(data,"features")
print(df)

dfg = df.groupby('properties.id').count()
print(dfg)
# using matplotlib scatter chart
plt.bar(dfg.index, dfg['properties.value'], width=0.5, color="maroon")
plt.show()

# using pandas plot
dfg.plot()

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json



#df = pd.DataFrame(data=data)
#print(df)

# using pandas for serializing
df= pd.json_normalize(data,"features")
x = df['properties.end_measure']
y = df['properties.value']
  
plt.plot(x,y)
plt.show()